# Shortest Distance to Coastline
> A visual guide using geopandas, shapely and folium
> 
I was given the data of a Bicycle company, on analysing it I found out people on the coastlines are more likely to buy a bicycle than people who live in the interior. So I tried to find out the shortest distance to the coastline from each point in the dataset.

This would give me a really nice feature to add to my model.

Read more about the case study [here]()

In [ ]:
# Importing the Dataset

import pandas as pd
import numpy as np

GeoCustomers = pd.read_csv('GeoCustomers.csv')
GeoCustomers.head()

In [ ]:
# Create a new dataframe with only the columns we need

df = GeoCustomers[['customer_id', 'lat', 'long']]
df.head()

# Getting the world map ready

In [ ]:
# Getting australia map from geopandas
import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
country = world[world.name == "Australia"]

# Plotting using folium

import folium
m_1 = folium.Map(location=[-25.2744, 133.7751], tiles='cartodbpositron', zoom_start=4)
folium.GeoJson(country).add_to(m_1)
m_1

In [ ]:
# Plotting only the coastline

coastline = country.boundary
m_2 = folium.Map(location=[-25.2744, 133.7751], tiles='cartodbpositron', zoom_start=4)
folium.GeoJson(coastline).add_to(m_2)
m_2

# Explaining the Implementation
- use shapely to create a point from the lat and long
- use shapely.nearst_points to find the nearest point on the coastline
- calculate the distance between the two points using harvesine distance

In [ ]:
# Example of how we it actually works

from shapely.ops import Point, nearest_points

point = Point(df["long"][0], df["lat"][0])  # longitude, latitude of base point
nearest = nearest_points(point, coastline)[1]  # nearest point on boundary

m_3 = folium.Map(location=[-33.934109, 151.264326], tiles="cartodbpositron", zoom_start=10)
folium.Marker([df["lat"][0], df["long"][0]]).add_to(m_3)
folium.Marker([nearest.y, nearest.x]).add_to(m_3)
folium.GeoJson(coastline).add_to(m_3)
m_3

Works perfectly, now we will calculate the distance for each point in the dataset

In [ ]:
# Calculating the distance between two points in km

import haversine as hs
from haversine import Unit

geo = Point(df['long'][0], df['lat'][0])
cos = Point(nearest.x, nearest.y)

# Calculate the distance using haveversine formula
loc1 = (geo.y, geo.x)
loc2 = (cos.y, cos.x)

result = hs.haversine(loc1, loc2, unit=Unit.KILOMETERS)
print(result)

# Writing the function

In [ ]:
from shapely.ops import nearest_points
import haversine as hs
from haversine import Unit


def coastline_minima(df, lat_col, long_col, country):
    """
    This function takes in a dataframe with latitude and longitude columns and returns the nearest point on the coastline and the distance in km

    Required libraries: geopandas, shapely, folium, haversine
    Returns: original dataframe with two new columns: nearest_point and distance_to_coast

    """
    # Getting the coastline
    world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    coastline = world[world.name.str.contains(country)].boundary

    # Creating empty lists to store the nearest point and the distance
    nearest_points_list = []
    distance_list = []

    for i in range(len(df)):
        geo = Point(df[long_col][i], df[lat_col][i])
        nearest = nearest_points(geo, coastline)[1]
        cos = Point(nearest.x, nearest.y)
        loc1 = (geo.y, geo.x)
        loc2 = (cos.y, cos.x)
        result = hs.haversine(loc1, loc2, unit=Unit.KILOMETERS)

        nearest_points_list.append(nearest)
        distance_list.append(result)

    # Adding the new columns to the dataframe
    df["nearest_point"] = nearest_points_list
    df["distance_to_coast"] = distance_list

    return df

In [ ]:
coastline_minima(df, 'lat', 'long', 'Australia')

Works perfectly fine


# References
- geopandas: [link](https://geopandas.org/)
- shapely: [link](https://shapely.readthedocs.io/)
- folium: [link](https://python-visualization.github.io/folium/)
- haversine: [link](https://pypi.org/project/haversine/)

